Demography 88<br>
Fall 2017<br>
Carl Mason (cmason@berkeley.edu)<br>
# Lab 8: The Mariel Boatlift: a "*Natural Experiment*"

This week and next, we will explore the labor market effects of the Mariel Boatlift.  David Card (world famous economist who happens to teach at Berkeley) saw the boatlift as a "natural experiment" -- in which the supply of labor in Miami was increased suddenly, unexpectedly and in some important ways: randomly.  In his famous 1990 paper,  "The Impact of the Mariel Boatlift on the Miami Labor Market", Card sets out to measure the impact of this labor market change on wages and employment status of pre-boatlift residents of Miami.

This week and next, we will first replicate and then modestly extend Card's paper.  Of course, this would be an excellent time to have **already** read the paper -- which is available on jstor as well as at: [http://davidcard.berkeley.edu/papers/mariel-impact.pdf]

The key substantive issue is whether and to what extent an increase in the labor supply leads to a reduction in wages of those previously resident.  This is precisely the question that we addressed in our discussion of Clemen's trillion dollar bills and the simple neoclassical model of immigration depicted in this totally viral app: http://shiny.demog.berkeley.edu/carlm/EconImmig0/,

The key technical challenge is to bootstrap a statistic that is  more complicated than a mean -- and the incorporation of sample weights. In many ways this project is structurally very similar to the Wage Impact lab from last week, except that it is a bit more complicated.

What we are going to ultimately do is called a "difference in differences" estimate.  We'll get into details of that next week, but what you need to keep in mind this week is that we're going to compute the *difference* between the mean earnings of a well defined group of workers in Miami and a similar group of workers in a "control" group of similar cities. We're going to do that difference computation at a time *before* the Mariel boatlift and again a few years *after* the boatlift.  The we'll compare the two differences.  If the difference got bigger(or smaller)  we can attribute that change to the Mariel boatlift.  Essentially, we will assume that anything important (but unrelated to the Mariel boatlift) that happened during that interval -- had an equal effect on both Miami and the control cities.  As usual, we are oversimplifying a bit -- but the gist of it is that the "difference in differences" framework, provides a way of "controlling" for all sorts of unobserved things.  

It is probably worth re-reading the above paragraph, because if you get what it's saying,  this lab is going to make a lot more sense.


The steps involved are:
1. Recreate tables 1, 3 and 4 of Card's famous paper.
1. Draw some graphs to illustrate the quantities (differences) in question.
2. Compare our findings with Card's regression based  outcomes.
3. Use bootstrap to quantify our uncertainty about the effects on the labor market disaggregated by education and occupation -- tests which are not part of the original paper.

We will accomplish steps 1&2 this week and 3&4 next week.

The data that we will use is of course the same as that used by Professor Card: the "Monthly Outgoing Rotation Groups" from the Current Population Survey (CPS). The CPS is a venerable survey which has been consistently conducted since unemployment was invented in the 1930s.  The survey is done every month by the Census Bureau for the Bureau of Labor Statistics and its purpose is to measure unemployment.  To do so  it asks a series of questions which come down to: "Do you have a job" and if not "Are you looking for one".  The latter determines whether or not the respondent is "in the labor force" and the former whether or not the person is employed. 

Of course, because surveys of this sort are quite expensive, the CPS does not merely ask the Employed?/Looking? questions but rather administers an impressive battery of questions on a wide range of topics including things closely related to employment such as  education and occupation, but also on topics of broader  interest to academics and policy makers-- such as computer and internet use; health insurance; tobacco use; veteran status; migration and civic engagement. Most of these questions are contained in what are known as monthly "supplements" to the "base survey" (which is concerned with employment).  Best of all, lots these data are available through IPUMS http://ipums.org. The data for this lab, however, were downloaded originally from the NBER site which makes the data available in a form much closer to what Professor Card used back in the Twentieth Century.
http://http://www.nber.org/data/morg.html.

The "outgoing rotation groups" (ORG) refers to the way the survey is conducted: each month a new group of respondents is selected. Each group is then polled for four consecutive months; then left alone for next eight months; and then polled again for four months. The income and employment questions in which we are interested, are asked *only* in the fourth and eighth interviews. This all means that:

1. Each respondent (household) is asked these questions two times during their 16 months of survey participation.
2. Each month only one eighth of currently empaneled CPS respondents are asked these questions.


In [ ]:
# Run this cell to import the stuff we'll need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')

%matplotlib inline
from datascience import Table
from datascience.predicates import are
from IPython.display import HTML, IFrame, display
datasite="http://courses.demog.berkeley.edu/mason88/data/"
quizsite="http://courses.demog.berkeley.edu/mason88/cgi-bin/quiz.py"
  
def cquiz(qno) : 
    import IPython, requests 
    try:
        sid
    except NameError: 
        print("HEY! did you enter your sid way up at the top of this notebook?")
    Linkit='{0}?qno={1}&sid={2}'.format(quizsite,qno,sid)
    #print(Linkit)
    html = requests.get(Linkit)
    #display(IFrame(Linkit, 1000, 300))
    display(IFrame(Linkit, 1000, 400))


    
######################
# Here it is ... the obvious place to put your student id
sid=""
######################
if sid == "" :
    print("HEY! didn't I tell you to put your sid in the obvious place")
 


In [ ]:
   
###
# put your student id in the obvious place below - or you will find yourself typing it quite frequently
###
sid=""

if sid == "" :
    print("HEY! didn't I tell you to put your sid in the obvious place")

In [ ]:
### #### #### ####
###  SELECTING  LAB  PARTNERS
### ### #### ####

N=24
numbers=np.arange(N)
print(np.mod(N,2))
if (not np.mod(N,2) == 0) :
    numbers=np.append(numbers,"lucky")
    N+=1
numbersTab=Table().with_column('n',numbers)
randomized=numbersTab.sample(k=N,with_replacement=False)
selection=randomized['n']
selection.shape = (2,int(N/2))
Table().with_columns('zero',selection[0],'one',selection[1]).show()

In [ ]:
## With whom are you working today?
cquiz('mariel-01partners')

In [ ]:
# Read the data -- "morg" stands of "merged outgoing rotation groups" which are a subset of the data collected in 
# Current Population Survey

morg=Table.read_table(datasite+"morgClean.csv")

# and add the variable lnWage for log of hourly wage
morg=morg.with_column('lnWage',np.log(morg['Earnhr']))

print(np.min(morg['age']))
print(np.max(morg['age']))
morg

## Recreating Table 1

Table 1 in Card's paper,  gives some descriptive statistics for Miami in 1979 before the Mariel Boatlift happens.

Recreating it will give us a feel for both the structure of the data and the situation in Miami in 1979.

Because the CPS is responsible for producing an unemployment statistic for each and every state, county and SMSA-- by race, ethnicity and other demographic characteristics, a *simple random sample* turns out **not** to be the best way to do it.  Collecting these data are expensive, and in order to be assured of having enough observations in **each** geographical/demographic category, a simple random sample would have to be very large (expensive).  The way this is avoided is with a "stratified" random sample, which is a type of "probability sample" (as described: https://www.inferentialthinking.com/chapters/08/5/sampling.html) .  To oversimplify a bit,  a stratified random sample contains a certain number of observations chosen at random from *each*  geographical area of interest. Then weights are calculated to adjust statistics that are computed from those data.  In order to calculate theses weights it is necessary to know each observation's probability of selection.  In the simplest case, this is just a the number of observations collected from the area divided by the number of individuals in the area who could have been selected. In the case of the CPS, the sample is stratified in many more ways besides geography, so even within a region and year the weights vary across individuals. 

The way to use weights, in the present (typical) case, is to think of them as the number of people in the universe who are *represented* by each individual in the sample.  Thus the number of people in area A =$\sum_{i\in{A}}{w_i}$ where $w_i$ is the sample weight of individual i and $i\in{A}$ indicates that individual i lives in area A.

Similarly, the weighted average of some characteristic, c, of people who live in area A would be given by: 
$$\frac{\sum_{i\in{A}}{(c_i*w_i)}}{\sum_{i\in{A}}{w_i}}$$

The above formulae are what David Card used to compute the figures in his Table 1.  It should also look pretty familiar--it is the same formula that we used in the Wage Impact lab.

In [ ]:
## So turning to Table 1
## Call this cell "code for panel 1"
## Here is some code to generate the top panel

## select observations from Miami in 1979

miami79=morg.where('SMSA',"Miami").where('year',1979)
# create columns of c_i * w_i  characteristic * weight
miami79.append_column('EducWT',miami79['Educ']*miami79['Weight3'])
miami79.append_column('LabfWT',miami79['Labf']*miami79['Weight3'])
# sum the c_i*w_i and the w_i  to get numerator and denominators from equation above
tab1_1=miami79.select(['EthRace','Weight3','EducWT','LabfWT']).\
    groups('EthRace',collect=np.nansum)

# relabel to remind ourselves that the sum of the wieghts is the number of observations, N
tab1_1.relabel('Weight3 nansum','N')
# weighted means per the equation above
tab1_1.append_column('EducWTM',tab1_1['EducWT nansum']/tab1_1['N'])
tab1_1.append_column('LabfWTM',tab1_1['LabfWT nansum']/tab1_1['N'])
tab1_1

# What do we think of these numbers ?

Profound discussion question: How close is close enough ?


## A note on weights

Constructing weights for survey samples is somewhat of a dark art.  While the principles are well understood, the execution invariably requires some judgement and compromise--mainly because the weighting always requires information external to the survey sample itself.  The CPS, for example, requires census data in order to know how many households exist in a particular county, as well as the demographic, racial and ethnic makeup of the region.  And if that were not enough, the Census is taken every 10 years, so  in years like 1979, the available census is quite out of date -- more judgement and external data are required in order to come up with estimates of 1979 population totals.

To get a flavor of what's involved, I half heartedly recommend taking a quick look at: https://www.census.gov/prod/2006pubs/tp-66.pdf.

The complexity/fragility of the weighting process is part of the reason that the CPS contain TWO weighting variables:  Weight3, that we have used thus far is one,  but there is also a column called 'Earnwt' which is a very slightly different weight that is supposed to be used for all earning related variables.  




In [ ]:
# scatter plot of two weighting variables
morg.scatter('Weight3','Earnwt',colors='year')

For your edification here are descriptions of the two weights from the NBER documentation:

weight:
>This is the Final Weight. The sum of the Final Weights in each monthly survey is the US non-institutional population. The CD-ROM excludes persons under 16 years of age. The outgoing rotation group includes one-fourth of that population. So one single month MORG file is one-fourth the population 16 years of age and over,and a year of MORG would sum to 3 times that population. Zero weights appear in some years, for records of unknown function. The implied two or four (1994 on) decimals on the tapes are explicit here. 1990-census-based weight for 2000-2002 are is
available as weightp.

earnwt:
>Earnings weight for all races. Used for tabulating earnings related items. Since the CD-ROM includes all persons asked earning questions, this sums to the total population each month and 12 times the population for each MORG file. This is not precisely 4 times the weight, presumably because the Census has external knowledge of the size and composition of the labor force. The implied decimals on the tapes are explicit here. A BLS letter suggests that this weight is preferred for all purposes. 1990-census-based earnwt for 2000-2002 is available as earnwtp. 


Fortunately, these details don't matter very much. I have experimented with both weights and (and with several other possible data adjustments and determined 2 things:

1. The weights matter very little
2. It is not possible to *precisely* reproduce Card's numbers despite having access to his computer code.




## Table 1  panel 2   (Code Required)

** Copy then modify the "code for panel 1" in order to produce panel 2 of Table 1**

Some notes:

1. 'Labf' is a True/False variable indicating whether the individual was in the labor force
2. 'age' is age in years those under 16 are not in the dataset
3. 'Educ' gives the number of years of school completed 
4. Your instructor believes that the weighted average number of years of education of non hispanic Blacks in Miami in 1979 was 11.7145.
4. When you have completed panel 2 of table 1 - click on the quiz link below to answer an easy question and get credit for your excellent work.

In [ ]:
## Copy "code for panel 1" into this cell and modify it to produce panel 2 in Card's Table 1
# See instructions in cell above
# 





tab1_2.show()

In [ ]:
cquiz('mariel-01')

In [ ]:

# Here's panel 3 of Table 1

# These numbers might be useful next week, we won't do anything with them this week, however.
tab1_3=miami79.where('Unemp',False).select(['EthRace','Occupation','Weight3']).\
    groups(['EthRace','Occupation'],collect=np.nansum)
tab1_3sums=miami79.where('Unemp',False).select(['EthRace','Weight3']).\
    groups(['EthRace'],collect=np.nansum)
# Here's the clever bit -- we need to divide by N in tab
tab1_3=tab1_3.join('EthRace',tab1_3sums,'EthRace')
tab1_3=tab1_3.with_column('Proportion',tab1_3['Weight3 nansum']/tab1_3['Weight3 nansum_2'])
print('Table 1 panel 3: Miamai 1979 occumpational distribution')
tab1_3.pivot('EthRace','Occupation',values='Proportion',collect=np.array)


In [ ]:
# Let's remind ourselves of how the morg dataset works
cquiz('mariel-02')

## Recreating Table 3  log(hourly wage)

Let's turn now to Table 3. Below is some code to generate an UNWEIGHTED version of the table.
Your job is to modify the code to include weights.  

#### It turns out that there are two sets of weights in the CPS.  For our purposes which we use does not matter much, but for reasons that I will bore you with later, we will use 'Earnwt' instead of 'weight3' from now on.


In [ ]:
## The unweighted version of Table 3
#for convenience a column that distinguishes Miami from the Control Cities
morg.append_column('Miami',morg['SMSA'] =='Miami')
# take the means of the lnWage for each important subgroup Miam/EthRace/year
tab3_UW=morg.select(['Miami','EthRace','year','lnWage']).groups(['Miami','EthRace','year'],np.nanmean)
# rename the columns for a more pleasing table
tab3_UW=tab3_UW.relabel('lnWage nanmean','meanLnWage')
print("UNWEIGHTED Table 3")
tab3_UW.show(10)

## Now pivot table trick to reformat the table so that it is easier to compare to the one in the paper
# 
# since there is ONE unique value for each category any "collect" function will work function

print('Miami')
tab3_UW.where('Miami',True).pivot('year','EthRace',values='meanLnWage',collect= np.array).show()
print('Control')
tab3_UW.where('Miami',False).pivot('year','EthRace',values='meanLnWage',collect=np.array).show()

These results, without weighting are reasonably close but we should be able to get closer to what Card found if we use   weights

## NOW  produce the *weighted* version of Table 3  (Coding Required)

The same principles apply in Table 3 as in Table 1 as far as weights are concerned.
Below are a couple of lines to get you started. 

Notes:
1. One gotcha here is that there are some observations which have a nonzero Earnwt but for which Earnhr is missing.  I am not sure how this happens -- possibly there are people whose earnings are all from capital ? whatever the reason -- they need to be dropped when computing the denominator, $\sum{w_i}$.
2. Your instructor believes that the weighted mean log hourly wage of of Non-Hispanic whites in Miami in 1983 was 1.779269 
3. Use Earnwt for the weighting variable



In [ ]:
# Table 3 with weights
# create a new column containing Earnhr multiplied by the weight variable
morg.append_column('lnWageWTD',np.log(morg['Earnhr'])*morg['Earnwt'])
#  create new column indicating quality of Earnhr because there are observations 
#  with positive Earnwt and  nan for Earnhr
#  The '~' operator is the 'not' operator; it inverts the boolean value of the expression that follows it
#  ~np.isnan == is NOT nan"
morg.append_column('GoodWageDat',~np.isnan(morg['Earnhr']))

## Code required here to complete the job of computing Table 3



tab3= ???  #(could require more than one line of code)




## pivot table trick to reformat tab3 to look like it does in the paper
print('Miami')
tab3.where('Miami',True).pivot('year','EthRace',values='meanLnWage',collect=np.array).show()
print('Control')
tab3.where('Miami',False).pivot('year','EthRace',values='meanLnWage',collect=np.array).show()

## How much difference do the weights make?

Discuss the histogram below.

Consider the following back of the envelope way to assess whether the difference between the weighted and unweighted values in Table 3 are big enough to matter:  The key question is would it matter (much) to the results if we used the un-weighted values in Table 3 in place of the weighted values. Without doing any computation consider the things that we are likely to compare:
1. Miami residents of a single EthRace across time.
1. Miami residents of a single EthRace in a single year vs people of the same category in a control city.
1. Residents of either place of a particular EthRace at a single year vs people of a different EthRace.

Scan the weighted Table 3 that you just computed and determine for each of the three cases above, what "large-ish" difference would be.  

Are each of these three "large-ish" values big or small compared to a large value in the histogram below:

In [ ]:
## Comparing the values with and without weights

# assuming your result from above is called tab3 and the mean log wage column is called 'meanLnWage'
Table().with_column('weighted-unweighted',tab3['meanLnWage']-\
        tab3_UW['meanLnWage']).hist(bins=25)


In [ ]:
cquiz("mariel-031")

In [ ]:
cquiz("mariel-04")

## Are the differences between your log wage estimates and Card's important? 






To inform your discussion... you may read in the values in Card's Table 3 by
executing the following cell an then creating a histogram just like the one above?


In [ ]:
## read data from Card's Table 3 which your instructor has thoughtfully typed for you
cardT3=Table.read_table('http://courses.demog.berkeley.edu/mason88/data/cardTable3.csv')
cardT3.append_column('Miami',cardT3['Miami'] =='yes')
cardT3.relabel('meanLnWage','cardMeanLnWage')
cardT3


In [ ]:
# Now to make the histogram comparing your results with Card's Table 3, we'll need to create a table wherein 
# one column holds Card's results and another holds your results. This should be a "simple" .join since cardT3
# and tab3 are similarly structured with columns for EthRace,year and Miami.

# It's tricker than it should be however, because .join only works on a single column -- and we would
# like to join rows based on three columns.  A reasonable solution is to create
## a variable, called 'rdx'  which is the concatination of EthRace,year and Miami

# surprisingly THIS does NOT work:
##cardT3.append_column(rdx,cardT3['EthRace']+cardT3['year'].astype(str))

#this works:
cardT3.append_column('rdx',np.char.add(
    np.char.add(cardT3['EthRace'],cardT3['year'].astype(str)),cardT3['Miami'].astype(str)))

tab3.append_column('rdx',np.char.add(
    np.char.add(tab3['EthRace'],tab3['year'].astype(str)),tab3['Miami'].astype(str)))
## and the join works
t3diff=cardT3.select(['rdx','cardMeanLnWage']).join('rdx',tab3)

# and the histogram
Table().with_column('Card-You',t3diff['cardMeanLnWage']-t3diff['meanLnWage']).hist(bins=25)



## In therory...

there should be no difference between the numbers that you have computed and those that David Card reported.  And yet,  there *are* small differences.  Given that we are using the same data set and that your instructor had access to Card's computer program,  this is kind of sad. It could certainly be that your instructor has lead you astray (that's happened before) but, of course I don't think that's the case. 

What has happened is that time has passed and even though that "monthly outgoing rotation groups" data from 1979-85 *should* not have changed since Card published his famous paper, it appears that that may not be the case.  It appears that small changes to the data are made by both the BLS and NBER (who agregates the data).  The changes are not enough to affect results, but it does emphasize how difficult it is to reproduce results -- even using the same data!

In [ ]:
cquiz('mariel-031')

# Summarize what we know so far with a few graphs

### The key question in this lab and the next is: to what extent, if any, were workers who lived in Miami in 1979 made worse (or better) off *as a result of* the influx of Cuban immigrants in 1980. Let's draw some graphs that compare wage trajectories of subgroups of Miami workers with similar people in the control group cities.

We'll start by writing a function that will do the calculations that we have been doing all day -- taking weighted averages of a value grouped by other characteristcs such as year, Miami residence and EthRace. Then we'll use the output to draw some useful graphs. The function, which will be quite useful in next week's exercise,  is provided, the graphs will require some coding.


In [ ]:
## The function is essentially a generalization of the code that you have already written
## it is worth understanding how it works as we will use it again
def wtdMean(data,depvar,gvars=['Miami','year'],wvar='Earnwt'):

    """
    given a dataset (generally morg) a dependent variable column e.g. lnWage; a list of
    variables by which to group the data, gvars, and a weight column (Earnwt)
    returns a table with one row for each unique combination of gvars along  the corresponding
    weighted mean of depvar
    """
    # creat a list of columns of the input data that we need; we'll discard the rest
    # note that we need to copy rather than asign here as we still need gvars
    allvars=gvars.copy()
    allvars.append(depvar)
    allvars.append(wvar)
    #get rid of columns we don't need
    dset=data.select(allvars)
    #get rid of rows that we don't want
    dset=dset.where(~np.isnan(dset[wvar]))
    dset=dset.where(~np.isnan(dset[depvar]))
    # compute the numerator and denominator
    dset.append_column(depvar+'WTD',dset[depvar]*dset[wvar])
    result=dset.groups(gvars,collect=np.nansum)
    # and do the division
    result.append_column(depvar+"WTDmean",result[depvar+ 'WTD'' nansum']/result[wvar+' nansum'])
    # return the result tossing out some intermediate calcultions
    return(result.drop([depvar+" nansum",depvar+"WTD"+" nansum"]))

In [ ]:
## Question about the wtdMean() function
cquiz('mariel-07')

## An example using the wtdMean() function

In [ ]:
# Example 0 - Using the wtdMean() funtion 
# mean wage of employed people in Miami vs control towns by year
MiamixCntrl=wtdMean(morg.where('Unemp',False),'Earnhr')
MiamixCntrl

## Using the result of the wtdMean() function to contruct a graph

In [ ]:
MiamixCntrl.scatter('year','EarnhrWTDmean',colors='Miami')

In [ ]:
wtdMean(morg.where('Labf',True),'Unemp',gvars=['Miami','year','EthRace'])



In [ ]:
## Interpret the graph above
cquiz('mariel-06')

## Another example using the wtdMean() funtion  + graphs

In [ ]:

DepVar='Earnhr'
temp=wtdMean(morg.where('Unemp',False).where('EthRace','NonHisp:black'),DepVar)
temp.scatter('year',DepVar+'WTDmean',colors='Miami')

In [ ]:
DepVar='Earnhr'
temp=wtdMean(morg.where('Unemp',False).where('EthRace','NonHisp:black').where('age',are.between_or_equal_to(20,30)),DepVar)
temp.scatter('year',DepVar+'WTDmean',colors='Miami')

# Congratulations you have completed this week's lab.

In [ ]:
cquiz('suro417')

In [ ]:
cquiz('mariel0-eval')